<img src="trump.png">

In [ ]:
! pip install pandas
! pip install -U scikit-learn

In [1]:
import pandas as pd
import sklearn.feature_extraction.text as text
import sklearn.linear_model as linear_model

In [2]:
## Helper functions
def clean_data(data):
    data['created_at']=pd.to_datetime(data['created_at'])
    data_sub=data.query("source=='Twitter for Android' or source=='Twitter for iPhone'")
    return data_sub
def train_test(data):
    test=data.query("created_at>'2016-11-08'")
    train=data.query("created_at<='2016-11-08'")
    y_train=train['source']
    y_test=test['source']
    y_train=y_train.map({'Twitter for Android':1,'Twitter for iPhone':0})
    y_test=y_test.map({'Twitter for Android':1,'Twitter for iPhone':0})
    X_train=train['text']
    X_test=test['text']
    return X_train,X_test,y_train,y_test
def get_tidf(X_train,X_test):
    tfidf=text.TfidfVectorizer(X_train.tolist())
    X_train_tfidf=tfidf.fit_transform(X_train.tolist())
    X_test_tfidf=tfidf.transform(X_test.tolist())
    return X_train_tfidf,X_test_tfidf,tfidf
def model(X_train,y_train):
    clf=linear_model.LogisticRegression()
    clf.fit(X_train,y_train)
    return clf
def make_prediction(clf,tfidf,tweet):
    p=tfidf.transform([tweet])
    prob=clf.predict_proba(p)[:,1].item()
    if prob>0.75:
        print("Tweet is by trump")
    else:
        print("Tweet is not by trump")

## Get data

In [3]:
data=pd.read_csv("https://s3.us-east-2.amazonaws.com/datafaculty/trump_tweets.csv")
data.head()

,created_at,favorite_count,id_str,in_reply_to_user_id_str,is_retweet,retweet_count,source,text
0,2015-12-31 23:21:49,16495,682703233492619264,NaN,False,6776,Twitter for Android,I would like to wish everyone A HAPPY AND HEAL...
1,2015-12-31 23:11:35,6824,682700657304989696,NaN,False,2755,Twitter for Android,"Do you believe that The State Department, on N..."
2,2015-12-31 18:51:12,6047,682635132038176768,NaN,False,2468,Twitter for iPhone,THANK YOU ILLINOIS! Let's not forget to get fa...
3,2015-12-31 16:52:38,8416,682605293856002048,NaN,False,2080,Twitter for iPhone,"HAPPY BIRTHDAY to my son, @DonaldJTrumpJr! Ver..."
4,2015-12-31 15:07:18,5780,682578783912759296,NaN,False,1875,Twitter for Android,I would feel sorry for @JebBush and how badly ...


#### Clean the data

In [4]:
data=clean_data(data)

#### Get train and test based on 2016 presidential elections

In [5]:
X_train,X_test,y_train,y_test=train_test(data)

#### Represent data as tfidf matrix

In [6]:
X_train_tfidf,X_test_tfidf,tfidf=get_tidf(X_train,X_test)

#### Build a classifier

In [8]:
mod=model(X_train_tfidf,y_train)

In [9]:
tweet='''Heading back home after an amazing inclusive conference about my favorite 
programming language R #rstudioconf . #rladies  you all rock. Plus, my first PR 
for dplyr #tidyversedevday #tidyverse. 😄'''

In [10]:
make_prediction(mod,tfidf,tweet)

Tweet is not by trump


In [11]:
tweet_t='''Without a Wall our Country can never have Border or National Security. 
With a powerful Wall or Steel Barrier, Crime Rates (and Drugs) will go substantially 
down all over the U.S. The Dems know this but want to play political games. 
Must finally be done correctly. No Cave!'''

In [12]:
make_prediction(mod,tfidf,tweet_t)

Tweet is by trump
